In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neural_network import MLPClassifier

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Data

In [3]:
train = pd.read_csv("purchase600-100cls-15k.lrn.csv")
test = pd.read_csv("purchase600-100cls-15k.tes.csv")

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 448 entries, ID to 446
dtypes: int64(448)
memory usage: 13.7 MB


In [5]:
train.head()

,ID,class,1,2,3,4,5,6,7,8,...,437,438,439,440,441,442,443,444,445,446
0,0,11,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,8,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Columns: 447 entries, ID to 446
dtypes: int64(447)
memory usage: 3.4 MB


In [7]:
test.head()

,ID,1,2,3,4,5,6,7,8,9,...,437,438,439,440,441,442,443,444,445,446
0,4000,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,4001,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,4002,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4003,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
X_train = train.drop(["ID", "class"], axis=1)
y_train = train["class"]

X_test = test.drop(["ID"], axis=1)

In [13]:
# Appearence of each class
y_train.value_counts().sort_index()

1     132
2     141
3     124
4     135
5      80
6     150
7      90
8     250
9     114
10    173
11    149
12    142
13    103
14     99
15    171
16     83
17    144
18    103
19    146
20    211
21    181
22     98
23    128
24    133
25    100
26    119
27    126
28    127
29    113
30    135
Name: class, dtype: int64

# Model selection

In [9]:
# Grid Search

cv = StratifiedKFold(5, shuffle=True, random_state=1987)
verbose = 3

## Multi-layer Perceptron

In [11]:
parameters = {
    "hidden_layer_sizes":[(1000), (1500), (2000), (2500), (3000), (3500), (4000), (4500), (5000), (5500), (6000)],
    "solver":["lbfgs"],
    "activation":["relu"],
    "alpha":[0.001, 0.01]}
model = GridSearchCV(MLPClassifier(), parameters, cv=cv, verbose=verbose, scoring="accuracy")
model.fit(X_train, y_train)

results = pd.DataFrame(model.cv_results_)
results = results[["param_hidden_layer_sizes", "param_alpha", "mean_test_score", "std_test_score"]]
results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 5 folds for each of 26 candidates, totalling 130 fits
[CV 1/5] END activation=relu, alpha=0.001, hidden_layer_sizes=2000, solver=lbfgs;, score=0.760 total time=  11.0s
[CV 2/5] END activation=relu, alpha=0.001, hidden_layer_sizes=2000, solver=lbfgs;, score=0.775 total time=  10.4s
[CV 3/5] END activation=relu, alpha=0.001, hidden_layer_sizes=2000, solver=lbfgs;, score=0.751 total time=  10.2s
[CV 4/5] END activation=relu, alpha=0.001, hidden_layer_sizes=2000, solver=lbfgs;, score=0.730 total time=  10.0s
[CV 5/5] END activation=relu, alpha=0.001, hidden_layer_sizes=2000, solver=lbfgs;, score=0.774 total time=  10.2s
[CV 1/5] END activation=relu, alpha=0.001, hidden_layer_sizes=2500, solver=lbfgs;, score=0.767 total time=  12.6s
[CV 2/5] END activation=relu, alpha=0.001, hidden_layer_sizes=2500, solver=lbfgs;, score=0.757 total time=  13.1s
[CV 3/5] END activation=relu, alpha=0.001, hidden_layer_sizes=2500, solver=lbfgs;, score=0.765 total time=  13.9s
[CV 4/5] END activation=re

,param_hidden_layer_sizes,param_alpha,mean_test_score,std_test_score
14,2500,0.01,0.78050,0.017421
15,3000,0.01,0.78050,0.012160
18,4500,0.01,0.77850,0.012634
23,7000,0.01,0.77850,0.014478
20,5500,0.01,0.77725,0.018173
19,5000,0.01,0.77650,0.010765
16,3500,0.01,0.77600,0.016477
24,7500,0.01,0.77575,0.014887
25,8000,0.01,0.77525,0.017826
22,6500,0.01,0.77500,0.011806


# Final model

In [11]:
best_model = model.best_estimator_
best_model.fit(X_train, y_train)

MLPClassifier(alpha=0.01, hidden_layer_sizes=(400,), solver='lbfgs')

In [12]:
predictions = pd.DataFrame(test["ID"])
predictions["class"] = best_model.predict(X_test)

In [13]:
predictions.to_csv("submission.csv", index=False)